In [ ]:
import cv2
import os

# Dimensions for the saved images
image_x, image_y = 64, 64

# Create folders for the dataset
def create_folder(folder_name):
    base_path = 'dataset'
    training_path = os.path.join(base_path, 'training_set', folder_name)
    test_path = os.path.join(base_path, 'test_set', folder_name)

    os.makedirs(training_path, exist_ok=True)
    os.makedirs(test_path, exist_ok=True)

# Capture images using the webcam
def capture_images(gesture_name):
    # Create folders for the specified gesture
    create_folder(gesture_name)

    # Webcam setup
    cam = cv2.VideoCapture(0)
    cv2.namedWindow("Gesture Capture")

    # Image counters
    training_set_counter = 1
    test_set_counter = 1
    total_images = 0

    while total_images < 150:  # Total of 150 images
        ret, frame = cam.read()
        if not ret:
            print("Failed to capture video. Exiting...")
            break

        # Flip the frame for better interaction
        frame = cv2.flip(frame, 1)

        # Define a region of interest (ROI) for capturing hand gestures
        roi = cv2.rectangle(frame, (100, 100), (300, 300), (0, 255, 0), 2)
        roi_cropped = frame[102:298, 102:298]

        # Preprocess the ROI
        gray = cv2.cvtColor(roi_cropped, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (7, 7), 0)
        mask = cv2.adaptiveThreshold(
            blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2
        )

        # Display the frames
        cv2.imshow("Gesture Capture", frame)
        cv2.imshow("Processed ROI", mask)

        # Wait for user input
        key = cv2.waitKey(1)

        if key == ord('x'):  # Save image when 'x' is pressed
            save_img = cv2.resize(mask, (image_x, image_y))

            # Decide whether to save in the training or test set
            if total_images < 120:  # First 120 images go to training
                img_path = f'dataset/training_set/{gesture_name}/{training_set_counter}.png'
                training_set_counter += 1
            else:  # Last 30 images go to test
                img_path = f'dataset/test_set/{gesture_name}/{test_set_counter}.png'
                test_set_counter += 1

            cv2.imwrite(img_path, save_img)
            print(f"Image saved: {img_path}")
            total_images += 1

        elif key == 27:  # Exit when 'Esc' key is pressed
            print("Exiting...")
            break

    print(f"Image capture complete. {training_set_counter - 1} images in training set, {test_set_counter - 1} in test set.")
    # Release the camera and close all windows
    cam.release()
    cv2.destroyAllWindows()

# Prompt user for gesture name
gesture_name = input("Enter the gesture name: ")
capture_images(gesture_name)
